# 6.1 Python Programming

In [12]:
import duckdb
conn = duckdb.connect("../week3/database.db")

#every cursor is doing one thing at a time
cur = conn.cursor()

In [3]:
# execute a command
cur.execute("SELECT * FROM Site LIMIT 5")

# return all the results
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

In [6]:
# return all results
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchall() # all results

[(1547,)]

In [7]:
# return first results
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

(1547,)

In [8]:
# return only value
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()[0]

1547

**NOTE:** Duckdb doesn't support iterators, but can use this work around:

In [9]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
    # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


**EXAMPLE:** Use a query to find the number of nests per each species. Create a print statement

In [18]:
query = """
   SELECT COUNT(*) FROM Bird_nests
   WHERE Species = ?
"""
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():  # DuckDB workaround
    code = row[0]
    # NOT NEEDED! prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code])  # <-- added argument here
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()


Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


## Pandas
Can import Pandas and use SQL statements.

In [19]:
import pandas as pd

In [20]:
# read in any SQL table as a pandas df
df = pd.read_sql("SELECT * FROM Site", conn)
df.head(5)

,Code,Site_name,Location,Latitude,Longitude,Area
0,barr,Barrow,"Alaska, USA",71.300003,-156.600006,220.399994
1,burn,Burntpoint Creek,"Ontario, Canada",55.200001,-84.300003,63.000000
2,bylo,Bylot Island,"Nunavut, Canada",73.199997,-80.000000,723.599976
3,cakr,Cape Krusenstern,"Alaska, USA",67.099998,-163.500000,54.099998
4,cari,Canning River Delta,"Alaska, USA",70.099998,-145.800003,722.000000


## Cursors
Cursors are an object that mediates between submitting a query and getting the desired results. If we wanted to run several queries, we'd want more cursors. Cursors return a list.

__Example:__ Return all values

In [21]:
# recreate cursor
conn = duckdb.connect("../week3/database.db")
cur = conn.cursor()

# 6.2 Python Programming Cont.

**EXAMPLE:** Use a select statement

In [22]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

__EXAMPLE:__ Get all results. Cursor is streaming mechanism, does not store results.



In [23]:
# return all values
cur.execute("SELECT COUNT(*) FROM Bird_nests")
print(cur.fetchall())

# return single value
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

[(1547,)]


(1547,)

__NOTE:__ duckdb does not support iterators

__NOTE:__ Avoid using `SELECT * FROM` since you are relying on the selection of rows. To create a more "stable"

__EXAMPLE:__ How many nests do we have for each species?

Approach 1: First get all species, then execute a count query for each species.

In [24]:
# create a general query, where %s will be filled in with the sp code
query = " SELECT COUNT (*) FROM Bird_nests WHERE Species = '%s' "

# apply query accross all species
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Approach 2: Instead of using %s, use a ? in the query. Note the `cur2.execute()` information changes, and this code is slightly shorter.

In [25]:
# create a general query using ? instead of %s
query = " SELECT COUNT (*) FROM Bird_nests WHERE Species = ? "

# apply query accross all species
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    cur2 = conn.cursor()
    cur2.execute(query, [code])
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

cur.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests
